## Imports and setup

In [ ]:
import tensorflow as tf
import pandas as pd
from keras.models import Sequential
from keras.models import clone_model
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
import random

Using TensorFlow backend.


## Delay

### Read data delays

In [ ]:
df=pd.read_excel("FA_delay_MGK22nm_LW_50K_delay.xlsx")
del df['index']
dataset=df.values
df.head(10)

,lp1,wp1,lp2,wp2,lp3,wp3,lp4,wp4,lp5,wp5,lp6,wp6,lp7,wp7,lp8,wp8,lp9,wp9,lp10,wp10,lp11,wp11,lp12,wp12,lp13,wp13,lp14,wp14,ln1,wn1,ln2,wn2,ln3,wn3,ln4,wn4,ln5,wn5,ln6,wn6,ln7,wn7,ln8,wn8,ln9,wn9,ln10,wn10,ln11,wn11,ln12,wn12,ln13,wn13,ln14,wn14,delay_lh_nodeaco,delay_hl_nodeaco,delay_lh_nodebco,delay_hl_nodebco,delay_lh_nodecco,delay_hl_nodecco
0,2.700000e-08,2.910000e-07,2.200000e-08,8.800000e-08,2.200000e-08,2.900000e-07,2.700000e-08,2.280000e-07,2.300000e-08,2.890000e-07,2.300000e-08,6.800000e-08,3.100000e-08,2.640000e-07,3.300000e-08,1.000000e-07,2.700000e-08,2.140000e-07,2.900000e-08,9.200000e-08,2.800000e-08,1.380000e-07,2.400000e-08,1.460000e-07,2.500000e-08,2.660000e-07,2.900000e-08,1.630000e-07,3.100000e-08,2.630000e-07,2.800000e-08,2.810000e-07,2.600000e-08,8.000000e-08,2.400000e-08,1.930000e-07,2.500000e-08,2.400000e-08,3.000000e-08,1.750000e-07,2.200000e-08,3.800000e-08,2.200000e-08,2.300000e-07,3.100000e-08,3.100000e-08,2.500000e-08,3.260000e-07,2.700000e-08,3.020000e-07,2.600000e-08,1.890000e-07,3.100000e-08,9.400000e-08,2.500000e-08,8.200000e-08,3.118000e-11,1.509820e-11,8.997060e-12,1.831700e-11,1.159000e-11,2.461790e-11
1,3.000000e-08,9.900000e-08,2.900000e-08,9.800000e-08,2.400000e-08,3.800000e-08,2.200000e-08,1.180000e-07,2.700000e-08,2.110000e-07,2.300000e-08,1.910000e-07,2.700000e-08,1.560000e-07,3.000000e-08,7.000000e-08,3.000000e-08,3.090000e-07,3.100000e-08,2.680000e-07,3.300000e-08,1.110000e-07,3.200000e-08,5.000000e-08,2.500000e-08,2.940000e-07,2.600000e-08,6.900000e-08,3.000000e-08,7.900000e-08,3.300000e-08,7.500000e-08,2.500000e-08,1.220000e-07,2.600000e-08,1.130000e-07,3.300000e-08,2.200000e-07,2.900000e-08,4.900000e-08,2.500000e-08,9.000000e-08,3.200000e-08,1.940000e-07,2.600000e-08,1.600000e-07,3.200000e-08,3.200000e-07,2.300000e-08,4.000000e-08,2.900000e-08,2.590000e-07,2.500000e-08,5.500000e-08,3.200000e-08,7.400000e-08,7.582970e-12,9.683250e-11,1.325560e-11,2.924000e-11,1.140300e-11,8.941500e-11
2,2.200000e-08,5.100000e-08,2.800000e-08,2.680000e-07,2.500000e-08,1.950000e-07,2.500000e-08,1.340000e-07,2.600000e-08,6.000000e-08,3.200000e-08,1.310000e-07,2.500000e-08,2.310000e-07,2.200000e-08,2.400000e-08,2.400000e-08,3.260000e-07,2.500000e-08,1.380000e-07,2.300000e-08,1.870000e-07,3.000000e-08,2.570000e-07,3.100000e-08,2.650000e-07,2.700000e-08,1.640000e-07,2.600000e-08,8.300000e-08,2.500000e-08,1.210000e-07,3.300000e-08,2.790000e-07,3.300000e-08,3.290000e-07,3.200000e-08,9.100000e-08,2.200000e-08,2.150000e-07,2.200000e-08,1.660000e-07,2.700000e-08,1.580000e-07,2.300000e-08,2.400000e-08,2.600000e-08,8.900000e-08,3.200000e-08,2.250000e-07,2.500000e-08,2.420000e-07,3.100000e-08,8.800000e-08,2.300000e-08,7.200000e-08,1.397910e-11,5.646650e-11,9.010820e-12,7.129340e-11,1.189230e-11,2.106490e-11
3,2.900000e-08,3.070000e-07,2.900000e-08,1.530000e-07,2.500000e-08,1.710000e-07,2.700000e-08,9.600000e-08,3.200000e-08,2.660000e-07,2.500000e-08,9.400000e-08,2.300000e-08,9.700000e-08,2.900000e-08,1.660000e-07,2.700000e-08,1.780000e-07,2.800000e-08,2.690000e-07,3.200000e-08,3.190000e-07,2.500000e-08,2.780000e-07,2.900000e-08,1.140000e-07,2.600000e-08,1.220000e-07,2.800000e-08,1.940000e-07,3.200000e-08,2.320000e-07,2.900000e-08,2.160000e-07,2.700000e-08,6.700000e-08,3.100000e-08,1.450000e-07,3.200000e-08,2.940000e-07,2.400000e-08,7.400000e-08,2.300000e-08,2.890000e-07,2.800000e-08,2.730000e-07,2.600000e-08,3.190000e-07,2.700000e-08,1.480000e-07,2.600000e-08,3.220000e-07,3.100000e-08,1.930000e-07,3.300000e-08,2.740000e-07,2.089050e-11,1.549870e-11,1.121150e-11,3.298870e-11,1.178350e-11,2.053000e-11
4,2.300000e-08,3.160000e-07,2.500000e-08,2.650000e-07,2.900000e-08,1.300000e-07,2.200000e-08,5.600000e-08,2.600000e-08,3.400000e-08,3.200000e-08,2.570000e-07,3.100000e-08,2.440000e-07,3.200000e-08,3.200000e-07,3.100000e-08,1.180000e-07,2.300000e-08,1.870000e-07,2.600000e-08,1.600000e-07,3.000000e-08,8.400000e-08,2.200000e-08,2.410000e-07,3.200000e-08,2.570000e-07,3.300000e-08,1.310000e-07,2.800000e-08,2.010000e-07,2.5

In [ ]:
df.shape

(50000, 62)

### Training and testing model

In [ ]:
X = dataset[:40000,0:56]
y = dataset[:40000,56:]

In [ ]:
scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
scalarX.fit(X)
scalarY.fit(y)
X = scalarX.transform(X)
y = scalarY.transform(y)

In [ ]:
print(X.shape, y.shape)

(40000, 56) (40000, 6)


In [ ]:
model = Sequential()
model.add(Dense(48, input_dim=56, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(6, activation='relu'))
model.compile(loss='mse', optimizer='adam')
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
40000/40000 [==============================] - 3s 81us/step - loss: 0.0066
Epoch 2/100
40000/40000 [==============================] - 3s 80us/step - loss: 0.0043
Epoch 3/100
40000/40000 [==============================] - 3s 78us/step - loss: 0.0035
Epoch 4/100
40000/40000 [==============================] - 3s 79us/step - loss: 0.0027
Epoch 5/100
40000/40000 [==============================] - 3s 80us/step - loss: 0.0022
Epoch 6/100
40000/40000 [==============================] - 3s 80us/step - loss: 0.0019
Epoch 7/100
40000/40000 [==============================] - 3s 80us/step - loss: 0.0014
Epoch 8/100
40000/40000 [==============================] - 3s 79us/step - loss: 0.0011
Epoch 9/100
40000/40000 [==============================] - 3s 79us/step - loss: 0.0010
Epoch 10/100
40000/40000 [==============================] - 3s 79us/step - loss: 9.4757e-04
Epoch 11/100
40000/40000 [==============================] - 3s 79us/step - loss: 7.9289e-04
Epoch 12/100
40000/40000 [=======

In [ ]:
X_test = dataset[40000:,0:56]
y_test = dataset[40000:,56:]
X_test = scalarX.transform(X_test)
y_test = scalarY.transform(y_test)

y_trainpred = model.predict(X)
y_trainpred2 = scalarY.inverse_transform(y_trainpred)

y_testpred = model.predict(X_test)
y_testpred2 = scalarY.inverse_transform(y_testpred)

y_test2 = scalarY.inverse_transform(y_test)
y_2 = scalarY.inverse_transform(y)

In [ ]:
for i in range(6):
    r2_train = r2_score(y_trainpred2[:,i], y_2[:,i])
    r2_test = r2_score(y_testpred2[:,i], y_test2[:,i])
    print("Train R2 Score for "+df.columns[56+i]+" : "+str(r2_train))
    print("Test  R2 Score for "+df.columns[56+i]+" : "+str(r2_test))
    print()

Train R2 Score for delay_lh_nodeaco : 0.9802074274642395
Test  R2 Score for delay_lh_nodeaco : 0.9816328028707034

Train R2 Score for delay_hl_nodeaco : 0.9806807565507004
Test  R2 Score for delay_hl_nodeaco : 0.9788962888080164

Train R2 Score for delay_lh_nodebco : 0.980424503431146
Test  R2 Score for delay_lh_nodebco : 0.979918713535117

Train R2 Score for delay_hl_nodebco : 0.9877981732353083
Test  R2 Score for delay_hl_nodebco : 0.9874179140234296

Train R2 Score for delay_lh_nodecco : 0.9827810500432184
Test  R2 Score for delay_lh_nodecco : 0.9816579088655885

Train R2 Score for delay_hl_nodecco : 0.95519724119262
Test  R2 Score for delay_hl_nodecco : 0.955020970448656



In [ ]:
for i in range(6):
    mse_train = mean_squared_error(y_trainpred2[:,i], y_2[:,i])
    mse_test = mean_squared_error(y_testpred2[:,i], y_test2[:,i])
    print("Train MSE Score "+df.columns[56+i]+" : "+str(mse_train))
    print("Test  MSE Score "+df.columns[56+i]+" : "+str(mse_test))
    print()

Train MSE Score delay_lh_nodeaco : 2.113610498082188e-24
Test  MSE Score delay_lh_nodeaco : 2.1530450774992304e-24

Train MSE Score delay_hl_nodeaco : 1.7806330839348778e-23
Test  MSE Score delay_hl_nodeaco : 1.9425289110512465e-23

Train MSE Score delay_lh_nodebco : 2.0105589063117323e-24
Test  MSE Score delay_lh_nodebco : 2.0923861189763247e-24

Train MSE Score delay_hl_nodebco : 1.4462661699555053e-23
Test  MSE Score delay_hl_nodebco : 1.4763349846112938e-23

Train MSE Score delay_lh_nodecco : 1.8698988088426363e-24
Test  MSE Score delay_lh_nodecco : 2.0487001327886926e-24

Train MSE Score delay_hl_nodecco : 3.6613525510569443e-23
Test  MSE Score delay_hl_nodecco : 3.6900988816953383e-23



In [ ]:
for i in range(len(X_test)):
    print(y_test2[i, :])
    print(y_testpred2[i, :])    
    print()

Streaming output truncated to the last 5000 lines.
[9.0795305e-12 2.8525772e-11 2.1441342e-11 1.9431224e-11 7.6516883e-12
 2.6701806e-11]

[2.33911e-11 2.14771e-11 9.85645e-12 1.04275e-11 1.25573e-11 1.10813e-10]
[2.1719302e-11 2.2102264e-11 9.8414306e-12 9.6391367e-12 1.2619711e-11
 1.0997455e-10]

[1.19456e-11 1.75716e-11 6.25757e-12 1.05042e-10 6.99458e-12 1.63517e-11]
[1.2400760e-11 1.9648188e-11 5.0084303e-12 9.6882287e-11 6.0947436e-12
 1.9054086e-11]

[1.33390e-11 2.59918e-11 1.05681e-11 1.53324e-11 3.31603e-11 1.40794e-11]
[1.2504225e-11 2.5641812e-11 1.0430975e-11 1.6038431e-11 3.2537115e-11
 1.7259808e-11]

[1.07587e-11 6.21808e-11 1.25545e-11 1.44195e-10 1.28670e-11 4.93910e-11]
[1.00425163e-11 5.90529917e-11 1.29991165e-11 1.45980367e-10
 1.15327470e-11 5.26003893e-11]

[8.49637e-12 2.50985e-11 1.80706e-11 9.03670e-11 1.89055e-11 2.35537e-11]
[7.9618230e-12 2.7612659e-11 1.9198594e-11 9.3703378e-11 1.9102754e-11
 3.0395807e-11]

[2.33347e-11 2.11550e-11 1.54649e-11 1.83730e

## Leakage

### Read data

In [ ]:
filename = "FA_leakage_MGK22nm_LW_50K_Leakage.xlsx"
df=pd.read_excel(filename)
df.head(10)

,index,vin_a,vin_b,vin_c,lp1,wp1,lp2,wp2,lp3,wp3,lp4,wp4,lp5,wp5,lp6,wp6,lp7,wp7,lp8,wp8,lp9,wp9,lp10,wp10,lp11,wp11,lp12,wp12,lp13,wp13,lp14,wp14,ln1,wn1,ln2,wn2,ln3,wn3,ln4,wn4,ln5,wn5,ln6,wn6,ln7,wn7,ln8,wn8,ln9,wn9,ln10,wn10,ln11,wn11,ln12,wn12,ln13,wn13,ln14,wn14,leakage
0,1,0,0,0,2.700000e-08,2.910000e-07,2.200000e-08,8.800000e-08,2.200000e-08,2.900000e-07,2.700000e-08,2.280000e-07,2.300000e-08,2.890000e-07,2.300000e-08,6.800000e-08,3.100000e-08,2.640000e-07,3.300000e-08,1.000000e-07,2.700000e-08,2.140000e-07,2.900000e-08,9.200000e-08,2.800000e-08,1.380000e-07,2.400000e-08,1.460000e-07,2.500000e-08,2.660000e-07,2.900000e-08,1.630000e-07,3.100000e-08,2.630000e-07,2.800000e-08,2.810000e-07,2.600000e-08,8.000000e-08,2.400000e-08,1.930000e-07,2.500000e-08,2.400000e-08,3.000000e-08,1.750000e-07,2.200000e-08,3.800000e-08,2.200000e-08,2.300000e-07,3.100000e-08,3.100000e-08,2.500000e-08,3.260000e-07,2.700000e-08,3.020000e-07,2.600000e-08,1.890000e-07,3.100000e-08,9.400000e-08,2.500000e-08,8.200000e-08,0.000003
1,2,0,0,0,3.000000e-08,9.900000e-08,2.900000e-08,9.800000e-08,2.400000e-08,3.800000e-08,2.200000e-08,1.180000e-07,2.700000e-08,2.110000e-07,2.300000e-08,1.910000e-07,2.700000e-08,1.560000e-07,3.000000e-08,7.000000e-08,3.000000e-08,3.090000e-07,3.100000e-08,2.680000e-07,3.300000e-08,1.110000e-07,3.200000e-08,5.000000e-08,2.500000e-08,2.940000e-07,2.600000e-08,6.900000e-08,3.000000e-08,7.900000e-08,3.300000e-08,7.500000e-08,2.500000e-08,1.220000e-07,2.600000e-08,1.130000e-07,3.300000e-08,2.200000e-07,2.900000e-08,4.900000e-08,2.500000e-08,9.000000e-08,3.200000e-08,1.940000e-07,2.600000e-08,1.600000e-07,3.200000e-08,3.200000e-07,2.300000e-08,4.000000e-08,2.900000e-08,2.590000e-07,2.500000e-08,5.500000e-08,3.200000e-08,7.400000e-08,0.000002
2,3,0,0,0,2.200000e-08,5.100000e-08,2.800000e-08,2.680000e-07,2.500000e-08,1.950000e-07,2.500000e-08,1.340000e-07,2.600000e-08,6.000000e-08,3.200000e-08,1.310000e-07,2.500000e-08,2.310000e-07,2.200000e-08,2.400000e-08,2.400000e-08,3.260000e-07,2.500000e-08,1.380000e-07,2.300000e-08,1.870000e-07,3.000000e-08,2.570000e-07,3.100000e-08,2.650000e-07,2.700000e-08,1.640000e-07,2.600000e-08,8.300000e-08,2.500000e-08,1.210000e-07,3.300000e-08,2.790000e-07,3.300000e-08,3.290000e-07,3.200000e-08,9.100000e-08,2.200000e-08,2.150000e-07,2.200000e-08,1.660000e-07,2.700000e-08,1.580000e-07,2.300000e-08,2.400000e-08,2.600000e-08,8.900000e-08,3.200000e-08,2.250000e-07,2.500000e-08,2.420000e-07,3.100000e-08,8.800000e-08,2.300000e-08,7.200000e-08,0.000003
3,4,0,0,0,2.900000e-08,3.070000e-07,2.900000e-08,1.530000e-07,2.500000e-08,1.710000e-07,2.700000e-08,9.600000e-08,3.200000e-08,2.660000e-07,2.500000e-08,9.400000e-08,2.300000e-08,9.700000e-08,2.900000e-08,1.660000e-07,2.700000e-08,1.780000e-07,2.800000e-08,2.690000e-07,3.200000e-08,3.190000e-07,2.500000e-08,2.780000e-07,2.900000e-08,1.140000e-07,2.600000e-08,1.220000e-07,2.800000e-08,1.940000e-07,3.200000e-08,2.320000e-07,2.900000e-08,2.160000e-07,2.700000e-08,6.700000e-08,3.100000e-08,1.450000e-07,3.200000e-08,2.940000e-07,2.400000e-08,7.400000e-08,2.300000e-08,2.890000e-07,2.800000e-08,2.730000e-07,2.600000e-08,3.190000e-07,2.700000e-08,1.480000e-07,2.600000e-08,3.220000e-07,3.100000e-08,1.930000e-07,3.300000e-08,2.740000e-07,0.000004
4,5,0,0,0,2.300000e-08,3.160000e-07,2.500000e-08,2.650000e-07,2.900000e-08,1.300000e-07,2.200000e-08,5.600000e-08,2.600000e-08,3.400000e-08,3.200000e-08,2.570000e-07,3.100000e-08,2.440000e-07,3.200000e-08,3.200000e-07,3.100000e-08,1.180000e-07,2.300000e-08,1.870000e-07,2.600000e-08,1.600000e-07,3.000000e-08,8.400000e-08,2.200000e-08,2.410000e-07,3.200000e-08,2.570000e-07,3.300000e-08,1.310000e-07,2.800000e-08,2.010000e-07,2.500000e-08,2.790000e-07,2.700000e-08,2.120000e-07,2.800000e-08,4.400000e-08,3.200000e-08,2.450000e-07,2.800000e-08,2.570000e-07,2.400000e-08,9.400000e-08,2.900000e-08,1.630000e-07,2.800000e-08,2.900000e-07,2.900000e-08,2.790000e-07,2.500000e-08,1.790000e-07,2.500000e-08,7.500000e-08,2.300000e-08,7.300000e-

In [ ]:
del df['index']
del df['vin_a']
del df['vin_b']
del df['vin_c']

print(df.shape)

(400000, 57)


In [ ]:
df.tail(10)

,lp1,wp1,lp2,wp2,lp3,wp3,lp4,wp4,lp5,wp5,lp6,wp6,lp7,wp7,lp8,wp8,lp9,wp9,lp10,wp10,lp11,wp11,lp12,wp12,lp13,wp13,lp14,wp14,ln1,wn1,ln2,wn2,ln3,wn3,ln4,wn4,ln5,wn5,ln6,wn6,ln7,wn7,ln8,wn8,ln9,wn9,ln10,wn10,ln11,wn11,ln12,wn12,ln13,wn13,ln14,wn14,leakage
399990,2.800000e-08,1.370000e-07,3.300000e-08,2.550000e-07,3.000000e-08,3.230000e-07,2.700000e-08,7.100000e-08,2.200000e-08,1.950000e-07,2.900000e-08,8.000000e-08,2.500000e-08,9.300000e-08,3.300000e-08,1.810000e-07,2.500000e-08,8.100000e-08,3.100000e-08,8.700000e-08,2.800000e-08,2.170000e-07,2.900000e-08,1.690000e-07,3.200000e-08,1.950000e-07,2.900000e-08,1.310000e-07,3.000000e-08,3.200000e-08,2.900000e-08,9.300000e-08,2.300000e-08,3.010000e-07,2.400000e-08,2.890000e-07,2.800000e-08,1.710000e-07,2.600000e-08,2.890000e-07,3.100000e-08,8.000000e-08,3.200000e-08,2.670000e-07,2.200000e-08,2.630000e-07,3.200000e-08,2.250000e-07,2.700000e-08,2.720000e-07,2.200000e-08,2.770000e-07,3.300000e-08,1.520000e-07,3.200000e-08,2.330000e-07,0.000004
399991,2.500000e-08,2.350000e-07,3.300000e-08,3.000000e-07,2.500000e-08,2.780000e-07,3.000000e-08,2.610000e-07,2.900000e-08,1.340000e-07,2.600000e-08,1.380000e-07,2.300000e-08,2.770000e-07,2.700000e-08,1.880000e-07,2.500000e-08,1.350000e-07,2.700000e-08,2.950000e-07,2.300000e-08,2.990000e-07,2.600000e-08,2.390000e-07,3.200000e-08,3.270000e-07,2.400000e-08,2.520000e-07,3.000000e-08,3.220000e-07,2.800000e-08,2.790000e-07,3.300000e-08,2.570000e-07,2.600000e-08,2.470000e-07,3.100000e-08,1.260000e-07,2.400000e-08,1.360000e-07,2.900000e-08,6.500000e-08,3.300000e-08,3.040000e-07,2.800000e-08,2.400000e-07,2.800000e-08,1.200000e-07,3.000000e-08,1.580000e-07,2.500000e-08,1.520000e-07,2.700000e-08,2.850000e-07,2.600000e-08,2.900000e-07,0.000004
399992,2.900000e-08,1.690000e-07,2.600000e-08,9.400000e-08,2.600000e-08,7.000000e-08,2.900000e-08,1.790000e-07,3.000000e-08,2.360000e-07,2.900000e-08,1.870000e-07,2.700000e-08,1.860000e-07,2.700000e-08,3.270000e-07,2.600000e-08,2.080000e-07,3.300000e-08,5.700000e-08,2.500000e-08,2.290000e-07,3.100000e-08,2.790000e-07,2.200000e-08,3.500000e-08,3.100000e-08,2.000000e-07,3.100000e-08,2.580000e-07,2.400000e-08,3.260000e-07,2.300000e-08,8.100000e-08,2.400000e-08,1.810000e-07,2.600000e-08,1.780000e-07,2.200000e-08,2.690000e-07,2.400000e-08,1.890000e-07,3.300000e-08,9.000000e-08,2.400000e-08,7.400000e-08,2.500000e-08,1.360000e-07,2.600000e-08,2.350000e-07,2.600000e-08,3.230000e-07,3.200000e-08,3.800000e-08,2.800000e-08,6.000000e-08,0.000003
399993,2.900000e-08,1.590000e-07,2.300000e-08,2.190000e-07,2.500000e-08,2.290000e-07,2.800000e-08,3.050000e-07,2.200000e-08,3.170000e-07,2.200000e-08,1.810000e-07,3.200000e-08,2.540000e-07,2.800000e-08,7.800000e-08,2.200000e-08,6.300000e-08,2.300000e-08,1.050000e-07,3.100000e-08,2.440000e-07,2.200000e-08,1.910000e-07,3.300000e-08,2.320000e-07,3.100000e-08,3.000000e-08,3.000000e-08,1.690000e-07,2.900000e-08,1.580000e-07,2.900000e-08,6.300000e-08,2.800000e-08,1.420000e-07,3.200000e-08,4.600000e-08,2.300000e-08,3.100000e-08,2.800000e-08,2.020000e-07,2.800000e-08,1.940000e-07,2.800000e-08,2.820000e-07,2.300000e-08,1.390000e-07,2.300000e-08,2.090000e-07,2.400000e-08,3.040000e-07,2.900000e-08,2.660000e-07,3.200000e-08,3.250000e-07,0.000003
399994,2.800000e-08,8.300000e-08,2.900000e-08,1.120000e-07,2.200000e-08,2.880000e-07,3.100000e-08,2.400000e-08,3.100000e-08,1.520000e-07,2.900000e-08,1.270000e-07,2.800000e-08,2.000000e-07,2.400000e-08,1.400000e-07,2.200000e-08,8.300000e-08,2.500000e-08,3.120000e-07,2.700000e-08,2.750000e-07,3.200000e-08,1.140000e-07,2.800000e-08,2.410000e-07,2.600000e-08,2.990000e-07,3.200000e-08,1.650000e-07,2.600000e-08,2.690000e-07,3.300000e-08,1.600000e-07,3.000000e-08,3.220000e-07,3.100000e-08,1.620000e-07,2.400000e-08,1.660000e-07,3.300000e-08,7.000000e-08,3.200000e-08,3.020000e-07,3.200000e-08,1.860000e-07,2.600000e-08,6.200000e-08,3.100000e-08,9.400000e-08,2.900000e-08,2.250000e-07,2.700000e-08,1.550000e-07,2.300000e-08,2.450000e-07,0.000004
399995,2.300000e-08,2.25000

In [ ]:
dataset=df.values[:50000]
print(dataset.shape)
print(dataset[:2])

(50000, 57)
[[2.70000e-08 2.91000e-07 2.20000e-08 8.80000e-08 2.20000e-08 2.90000e-07
  2.70000e-08 2.28000e-07 2.30000e-08 2.89000e-07 2.30000e-08 6.80000e-08
  3.10000e-08 2.64000e-07 3.30000e-08 1.00000e-07 2.70000e-08 2.14000e-07
  2.90000e-08 9.20000e-08 2.80000e-08 1.38000e-07 2.40000e-08 1.46000e-07
  2.50000e-08 2.66000e-07 2.90000e-08 1.63000e-07 3.10000e-08 2.63000e-07
  2.80000e-08 2.81000e-07 2.60000e-08 8.00000e-08 2.40000e-08 1.93000e-07
  2.50000e-08 2.40000e-08 3.00000e-08 1.75000e-07 2.20000e-08 3.80000e-08
  2.20000e-08 2.30000e-07 3.10000e-08 3.10000e-08 2.50000e-08 3.26000e-07
  2.70000e-08 3.02000e-07 2.60000e-08 1.89000e-07 3.10000e-08 9.40000e-08
  2.50000e-08 8.20000e-08 2.79878e-06]
 [3.00000e-08 9.90000e-08 2.90000e-08 9.80000e-08 2.40000e-08 3.80000e-08
  2.20000e-08 1.18000e-07 2.70000e-08 2.11000e-07 2.30000e-08 1.91000e-07
  2.70000e-08 1.56000e-07 3.00000e-08 7.00000e-08 3.00000e-08 3.09000e-07
  3.10000e-08 2.68000e-07 3.30000e-08 1.11000e-07 3.20000e-08

### Train and test model

In [ ]:
X = dataset[:40000,0:56]
y = dataset[:40000,56:]

In [ ]:
scalarX2, scalarY2 = MinMaxScaler(), MinMaxScaler()
scalarX2.fit(X)
scalarY2.fit(y)
X = scalarX2.transform(X)
y = scalarY2.transform(y)

In [ ]:
print(X.shape, y.shape)

(40000, 56) (40000, 1)


In [ ]:
model2 = Sequential()
model2.add(Dense(48, input_dim=56, activation='relu'))
model2.add(Dense(32, activation='relu'))
model2.add(Dense(16, activation='relu'))
model2.add(Dense(1, activation='relu'))
model2.compile(loss='mse', optimizer='adam')
model2.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
40000/40000 [==============================] - 4s 89us/step - loss: 0.0037
Epoch 2/100
40000/40000 [==============================] - 4s 90us/step - loss: 9.5513e-04
Epoch 3/100
40000/40000 [==============================] - 4s 90us/step - loss: 9.3655e-04
Epoch 4/100
40000/40000 [==============================] - 4s 88us/step - loss: 8.7744e-04
Epoch 5/100
40000/40000 [==============================] - 3s 83us/step - loss: 8.0021e-04
Epoch 6/100
40000/40000 [==============================] - 3s 83us/step - loss: 6.6954e-04
Epoch 7/100
40000/40000 [==============================] - 3s 84us/step - loss: 5.2012e-04
Epoch 8/100
40000/40000 [==============================] - 3s 81us/step - loss: 4.4268e-04
Epoch 9/100
40000/40000 [==============================] - 3s 82us/step - loss: 3.8956e-04
Epoch 10/100
40000/40000 [==============================] - 3s 83us/step - loss: 3.5508e-04
Epoch 11/100
40000/40000 [==============================] - 3s 82us/step - loss: 3.0690e-04
E

In [ ]:
X_test = dataset[40000:,0:56]
y_test = dataset[40000:,56:]
X_test = scalarX2.transform(X_test)
y_test = scalarY2.transform(y_test)

y_trainpred = model2.predict(X)
y_trainpred2 = scalarY2.inverse_transform(y_trainpred)

y_testpred = model2.predict(X_test)
y_testpred2 = scalarY2.inverse_transform(y_testpred)

y_test2 = scalarY2.inverse_transform(y_test)
y_2 = scalarY2.inverse_transform(y)

In [ ]:
for i in range(1):
    r2_train = r2_score(y_trainpred2[:,i], y_2[:,i])
    r2_test = r2_score(y_testpred2[:,i], y_test2[:,i])
    print("Train R2 Score for "+df.columns[56+i]+" : "+str(r2_train))
    print("Test  R2 Score for "+df.columns[56+i]+" : "+str(r2_test))
    print()

Train R2 Score for leakage : 0.9949548034313874
Test  R2 Score for leakage : 0.9934270566550332



In [ ]:
for i in range(1):
    mse_train = mean_squared_error(y_trainpred2[:,i], y_2[:,i])
    mse_test = mean_squared_error(y_testpred2[:,i], y_test2[:,i])
    print("Train MSE Score "+df.columns[56+i]+" : "+str(mse_train))
    print("Test  MSE Score "+df.columns[56+i]+" : "+str(mse_test))
    print()

Train MSE Score leakage : 1.0675544311822307e-15
Test  MSE Score leakage : 1.385742462347896e-15



In [ ]:
for i in range(len(X_test)):
    print(y_test2[i, :])
    print(y_testpred2[i, :])    
    print()

Streaming output truncated to the last 5000 lines.

[3.32035e-06]
[3.317618e-06]

[3.72954e-06]
[3.7095756e-06]

[3.19436e-06]
[3.1683617e-06]

[3.0304e-06]
[3.0505742e-06]

[4.02585e-06]
[3.8932553e-06]

[3.37557e-06]
[3.3541176e-06]

[3.79608e-06]
[3.8570342e-06]

[2.85226e-06]
[2.864662e-06]

[3.63254e-06]
[3.6277834e-06]

[3.40138e-06]
[3.3932336e-06]

[3.05593e-06]
[3.0810102e-06]

[2.25868e-06]
[2.2843697e-06]

[3.62996e-06]
[3.6024242e-06]

[3.5937e-06]
[3.6151607e-06]

[3.48367e-06]
[3.4865202e-06]

[2.9015e-06]
[2.8849122e-06]

[2.50057e-06]
[2.433848e-06]

[2.96309e-06]
[2.956032e-06]

[2.84162e-06]
[2.8157822e-06]

[4.13696e-06]
[4.143222e-06]

[3.50563e-06]
[3.4848426e-06]

[2.79594e-06]
[2.7828244e-06]

[2.70652e-06]
[2.7212268e-06]

[2.97002e-06]
[2.9498326e-06]

[3.5615e-06]
[3.5368353e-06]

[3.28421e-06]
[3.2773603e-06]

[3.66888e-06]
[3.6249858e-06]

[3.19893e-06]
[3.2650773e-06]

[3.45385e-06]
[3.4796938e-06]

[3.2821e-06]
[3.2683854e-06]

[2.01952e-06]
[1.9732838e-06

## Converging to minima using genetic algorithm

In [ ]:
seed_data = dataset[0,0:56].tolist() # taking random value

GENE_LEN = len(seed_data)
ULIM_COEFF = 330 * 10**(-9) # w/l upper limit
LLIM_COEFF = 22 * 10**(-9) # w/l lower limit
POPULATION_SIZE = 10 # keep it even
step = 10**(-1)
ALLOWED_ACCESS = 10000 # access to model
ELITE_COUNT = int( POPULATION_SIZE * 0.2)
CROSS_COUNT = POPULATION_SIZE - ELITE_COUNT

DELAY_BOUND = 10**(-10)

In [ ]:
def give_order(num):
	'''
	returns the order of magnitude of number given
	'''
	num = abs(num)

	if num == 0.0:
		order = 0
	elif num >= 1.0:
		n = int(num)
		order = -1
		while n > 0:
			n = int(n/10)
			order += 1
	else:
		order = 0
		while num < 1.0:
			num *= 10
			order -= 1
	return order

In [ ]:
def mutate(num):
	'''
	num is number to mutate
	step is magnitude by which to mutate
	'''
	r = (random.random() - 0.5) * 2.0 # generates number [-1.0, 1.0]
	r *= step * 10**give_order(num)
	r += num

	if r > ULIM_COEFF:
		return ULIM_COEFF
	elif r < LLIM_COEFF:
		return LLIM_COEFF
	else:
		return r

In [ ]:
def calculate_fitness(x):
	'''
	Lower fitness value => more fit.
	Fittest at value 0.
	'''
	fitness_weights = np.array([+1.0, +1.0, +1.0, +1.0, +1.0, +1.0])  # for delay
	fitness_weights2 = np.array([+1.0]) # for leakage

	x = np.array(x) 
	x = np.expand_dims(x, axis=0)
	
	x1 = scalarX.transform(x)
	x2 = scalarX2.transform(x)

	y_pred = scalarY.inverse_transform(model.predict(x1))
	y_pred2 = scalarY2.inverse_transform(model2.predict(x2))

	ret_fitness = (np.sum(y_pred * fitness_weights)).item() + (np.sum(y_pred2 * fitness_weights2)).item()

	# loop for heavy penalty for breaking delay bound 
	penalty = 1.0
	for i in y_pred[0]:
		if i > DELAY_BOUND:
			ret_fitness += penalty
	
	return ret_fitness

In [ ]:
def calculate_rank(f):
	'''
	f is fitness
	sf is sorted fitness
	r contains ranks of f
	'''
	sf = sorted(f)
	r = [] # ranks
	for i in f:
		rnum = sf.index(i)
		while rnum in r: # To avoid duplicate ranks
			rnum += 1
		r.append(rnum)
	return r

In [ ]:
def initial_population():
	'''
	p is population
	'''
	mutation_probability = 0.4 # TUNE
	p = []
	p.append(seed_data)
	for i in range(POPULATION_SIZE-1):
		gene = seed_data[:]
		for i in range(GENE_LEN):
			if random.random() < mutation_probability:
				gene[i] = mutate(gene[i])
		p.append(gene)
	return p

In [ ]:
def calculate_chance(r, p):
	'''
	r is rank
	pop is population
	'''
	pc = 0.30
	c = []
	for i in range(POPULATION_SIZE):
		c.append(pc * (1-pc)**r[i])
	return c 

In [ ]:
def select_population(pop, c):
	'''
	pop is population
	c is chance
	'''
	return random.choices(pop, c, k=CROSS_COUNT)

In [ ]:
def do_crossover(s):
	'''
	s is selected population
	co is crossover
	p parent
	c child
	'''
	co = []
	for i in range(CROSS_COUNT):
		p1 = s[(i+0) % CROSS_COUNT]
		p2 = s[(i+1) % CROSS_COUNT]
		c = []
		for j in range(GENE_LEN):
			if random.random() < 0.5:
				c.append(p1[j])
			else:
				c.append(p2[j])
		co.append(c)

	return co

In [ ]:
def do_mutation(c):
	'''
	c is cross
	mc is mutated cross
	mg is mutated gene
	'''
	mutation_probability = 0.15

	mc = []
	for gene in c:
		mg = []
		for i in gene:
			if random.random() <= mutation_probability:
				mg.append(mutate(i))
			else:
				mg.append(i)
		mc.append(mg)

	global step
	step *= 0.9999 # keep less than 1.0
	
	return mc

In [ ]:
def pick_elite(population, rank):
	elite = []
	cur_rank = 0 # to avoid duplicates
	for i in range(ELITE_COUNT):
		elite.append(population[rank.index(cur_rank)])
		cur_rank += 1
		while(cur_rank < len(population) and population[rank.index(cur_rank)] in elite):
			cur_rank += 1
		if cur_rank >= len(population):
			cur_rank = 0 
	return elite

In [ ]:
def print_delays_leakage(x):
    '''
    prints the delay of optimized l and w
    '''
    x = np.array(x)
    x = np.expand_dims(x, axis=0)
    x1 = scalarX.transform(x)
    x2 = scalarX2.transform(x)

    y_pred = scalarY.inverse_transform(model.predict(x1))
    y_pred2 = scalarY2.inverse_transform(model2.predict(x2))
    
    print('optimised delays =', y_pred)
    print('optimised leakage =', y_pred2)

In [ ]:
fitest = [float('inf'), []] # [fitness_value, genes]
population = initial_population()

generation = 1
access_left = ALLOWED_ACCESS
while access_left > POPULATION_SIZE:
	fitness = []
	for i in population:
		fitness.append(calculate_fitness(i))

	rank = calculate_rank(fitness[:])	
	
	# print(fitness[rank.index(0)])
	if fitness[rank.index(0)] < fitest[0]:
		fitest[0] = fitness[rank.index(0)]
		fitest[1] = population[rank.index(0)][:]
		print('generation =', generation, ' fitness =', fitest[0])

	chance = calculate_chance(rank, population)
	select = select_population(population, chance)
	cross = do_crossover(select)
	elite = pick_elite(population, rank)
	population = do_mutation(cross) + elite
	
	access_left -= POPULATION_SIZE
	generation += 1

print('optimised input variables:', fitest[1])
print_delays_leakage(fitest[1])

generation = 1  fitness = 2.7985244952892507e-06
generation = 2  fitness = 2.777224899408806e-06
generation = 3  fitness = 2.768830252243039e-06
generation = 4  fitness = 2.7417778030785697e-06
generation = 6  fitness = 2.7164957363727885e-06
generation = 7  fitness = 2.704305175095338e-06
generation = 8  fitness = 2.7025408657092506e-06
generation = 9  fitness = 2.6820067363392958e-06
generation = 11  fitness = 2.6686376439185103e-06
generation = 12  fitness = 2.6541963721229272e-06
generation = 13  fitness = 2.6433698369467476e-06
generation = 14  fitness = 2.632447805067141e-06
generation = 15  fitness = 2.6086661100563e-06
generation = 16  fitness = 2.602592247638938e-06
generation = 17  fitness = 2.590897441016697e-06
generation = 18  fitness = 2.5877273780207377e-06
generation = 19  fitness = 2.5811881807129636e-06
generation = 20  fitness = 2.559644242042594e-06
generation = 21  fitness = 2.556306617696423e-06
generation = 22  fitness = 2.5448914570871256e-06
generation = 23  fi